In [5]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt

In [6]:
from load_data import load_and_transform_data

In [7]:
X_train, y_train, X_val, y_val, X_test, y_test = load_and_transform_data(SEED=42)

Dataset already exists at: chest_xray_data

Dataset ready at: c:\Users\eirik\Desktop\FYSSTK3155_Dataprocessing_and_MachineLearning\Project3_FYSSTK3155-4155\Code\chest_xray_data
Dataset already exists at: chest_xray_data_split



Loading test: 100%|██████████| 587/587 [00:04<00:00, 139.92it/s]


Train: (4684, 480, 480), Val: (585, 480, 480), Test: (587, 480, 480)


In [8]:
# Check shape of subsets
print(f"X train shape:\t", X_train.shape)
print(f"y train shape:\t", y_train.shape)
print(f"X val shape:\t", X_val.shape)
print(f"y val shape:\t", y_val.shape)
print(f"X test shape:\t", X_test.shape)
print(f"y test shape:\t", y_test.shape)

X train shape:	 (4684, 480, 480)
y train shape:	 (4684,)
X val shape:	 (585, 480, 480)
y val shape:	 (585,)
X test shape:	 (587, 480, 480)
y test shape:	 (587,)


In [ ]:
GRU = keras.Sequential()                          # Create a base sequential model
GRU.add(keras.Input(shape=(480, 480)))            # Set the input shape
GRU.add(layers.GRU(units=240, return_sequences=True))     # Add a simple RNN layer
GRU.add(layers.GRU(units=30, return_sequences=True))     # Add a simple RNN layer
GRU.add(layers.GRU(units=15))     # Add a simple RNN layer
GRU.add(layers.Dense(units=2, activation="softmax")) # Output layer

# Get a summary of model configuration
GRU.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                     │ (None, 480, 120)       │       216,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 480, 30)        │        13,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ (None, 15)             │         2,115 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            32 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 232,547 (908.39 KB)

 Trainable params: 232,547 (908.39 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
batch_size = 32
epochs = 10

In [17]:
GRU.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

GRU_trainer = GRU.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

GRU_test_loss, GRU_test_acc = GRU.evaluate(X_test, y_test)
print(f'GRU Test Accuracy: {GRU_test_acc}')

Epoch 1/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 23s 150ms/step - accuracy: 0.7252 - loss: 0.5858 - val_accuracy: 0.7316 - val_loss: 0.5773
Epoch 2/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.7827 - loss: 0.4691 - val_accuracy: 0.8359 - val_loss: 0.3635
Epoch 3/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 177ms/step - accuracy: 0.8561 - loss: 0.3335 - val_accuracy: 0.8855 - val_loss: 0.2688
Epoch 4/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 176ms/step - accuracy: 0.8918 - loss: 0.2666 - val_accuracy: 0.8855 - val_loss: 0.2668
Epoch 5/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 180ms/step - accuracy: 0.9031 - loss: 0.2392 - val_accuracy: 0.9060 - val_loss: 0.2367
Epoch 6/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 27s 181ms/step - accuracy: 0.9157 - loss: 0.2114 - val_accuracy: 0.9094 - val_loss: 0.2238
Epoch 7/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 26s 179ms/step - accuracy: 0.9236 - loss: 0.1954 - val_accuracy: 0.9111 - val_loss: 0.2326
Epoch 8/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 23s 155ms/step - accuracy: 0.9321 - loss: 0